In [28]:
import numpy as np
import pandas as pd
import mlflow

In [29]:
mlflow.set_tracking_uri('http://ec2-13-61-2-37.eu-north-1.compute.amazonaws.com:5000')
mlflow.set_experiment('Exp 5 - ML Algos with HP Tuning')

<Experiment: artifact_location='s3://neeraj-first-bucket/870835495118995563', creation_time=1736945300249, experiment_id='870835495118995563', last_update_time=1736945300249, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import ADASYN
import optuna

In [31]:
df = pd.read_csv('dataset.csv')
df.dropna(inplace=True)
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [32]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2)

# vectorizer
ngram_range = (1,2)
max_features = 2000
vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
X_train_trf = vectorizer.fit_transform(X_train)
X_test_trf = vectorizer.fit_transform(X_test)

# oversampling
adasyn = ADASYN(random_state=42)
X_train_trf, y_train = adasyn.fit_resample(X_train_trf, y_train)


In [33]:
def log_mlflow(model_name, model, X_train, y_train, X_test, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_Adasyn_Tfidf_Bigram")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algorithm", model_name)

        # model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # metrics
        accuracy = accuracy_score(y_test, y_pred)
        class_report = classification_report(y_test, y_pred, output_dict=True)

        mlflow.log_metric("accoracy", accuracy)
        for label, metrics in class_report.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [34]:
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)
    p = trial.suggest_categorical('p', [1, 2])

    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    return accuracy_score(y_test, model.fit(X_train_trf, y_train).predict(X_test_trf))

In [35]:
def optuna_run_experiment():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_knn, n_trials=30)

    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])
    
    log_mlflow("KNN", best_model, X_train_trf, y_train, X_test_trf, y_test)

In [36]:
optuna_run_experiment()

[I 2025-01-17 11:51:27,573] A new study created in memory with name: no-name-b4d86cd4-9f60-4943-abd1-5e977ea0491f
[I 2025-01-17 11:51:31,066] Trial 0 finished with value: 0.34651575071594165 and parameters: {'n_neighbours': 17, 'p': 1}. Best is trial 0 with value: 0.34651575071594165.
[I 2025-01-17 11:51:35,772] Trial 1 finished with value: 0.3426973953361516 and parameters: {'n_neighbours': 3, 'p': 2}. Best is trial 0 with value: 0.34651575071594165.
[I 2025-01-17 11:51:40,934] Trial 2 finished with value: 0.3211509614073367 and parameters: {'n_neighbours': 27, 'p': 2}. Best is trial 0 with value: 0.34651575071594165.
[I 2025-01-17 11:51:45,050] Trial 3 finished with value: 0.34597027137597164 and parameters: {'n_neighbours': 13, 'p': 1}. Best is trial 0 with value: 0.34651575071594165.
[I 2025-01-17 11:51:51,339] Trial 4 finished with value: 0.31555979817264423 and parameters: {'n_neighbours': 21, 'p': 2}. Best is trial 0 with value: 0.34651575071594165.
[I 2025-01-17 11:51:57,650] T

KeyError: 'n_neighbors'